ALS Recommender system example

This example uses the movielense data set - https://grouplens.org/datasets/movielens/ as input.

The data set has 3 columns:

* movieId
* rating
* userId

It has around 1500 records in which 29 users gave ratings for 45 to 50 movies each.

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('rec').getOrCreate()

Spark MLlib provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model (defaults to 10).
* maxIter is the number of iterations to run (defaults to 10).
* regParam specifies the regularization parameter (lambda) in ALS (defaults to 1.0).
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data (defaults to false which means using explicit feedback).
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0).
* nonnegative specifies whether or not to use nonnegative constraints for least squares (defaults to false).

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [ ]:
data.head()

In [ ]:
data.describe().show()

We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [ ]:
# Split the data 90:10
(training, test) = data.randomSplit([0.9, 0.1])

In [ ]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Evaluate the model using the test portion of the data 

In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [ ]:
predictions.show()

The RMSE described our error in terms of the stars rating column.

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Since we have the model now, let us use it to supply a recommendation to a user.

We are using the user id 11 and getting the details from the test data

In [ ]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [ ]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

In [ ]:
reccomendations = model.transform(single_user)

In [ ]:
reccomendations.orderBy('prediction',ascending=False).show()